<a href="https://colab.research.google.com/github/janiosl/python.ds/blob/master/py_acoes_workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Referência:

RODRIGUES, Paulo (2019). Python Aplicado: Bolsa de Valores - Um guia para construção de análises e indicadores. 

* Exemplos ajustado em função de atualização de pacotes e para melhor uso na plataforma Google Colab.
* Para uso de alguns gráficos é necessário a criação de uma conta gratuita ou paga da ferramenta Chart Studio (https://chart-studio.plotly.com).



In [1]:
#Instala o pacote para geração dos gráficos de análise de ações
!pip install chart-studio

     |████████████████████████████████| 71kB 3.5MB/s 


Importação das bibliotecas matemáticas, estatísticas e de geração de gráficos para nossa análise.

In [2]:
#Bibliotecas
import pandas as pd
import pandas_datareader as pdr
import datetime
import numpy as np
import plotly.graph_objects as go
from matplotlib import pyplot as plt
import seaborn as sns
import chart_studio.plotly as py

Para rodar a célula abaixo os dados deverão ser substituídos pelo seus dados de login na ferramenta Chart Studio.

In [3]:
#Login na ferramenta de geração de gráficos
#Substituir dados abaixo por seu usuário e api key gerados no Char-Studio
plotly_username = 'usuario'
plotly_key = 'api key'

py.sign_in(plotly_username, plotly_key)

In [4]:
def get(tickers, startdate, enddate):
  """
  Função criada por Paulo Rodrigues (2019)
  Realiza downoads dos dados a partir da base do Yahoo Finance
  Agrupa por acao e ordena por data
  """

  def data(ticker):
    #Download dos dados
    return (pdr.get_data_yahoo(ticker,
                               start=startdate,
                               end=enddate))
    
  #Aplica funcao de download aos papeis desejados
  datas = map(data, tickers)

  #Retorna os dados organizados
  return (pd.concat(datas,
                    keys=tickers,
                    names=['Ticker', 'Date']))

In [5]:
#Dados dos papeis e da consulta
tickers = ['GGBR4.SA', 'CIEL3.SA', 'BBDC4.SA']
start_date = datetime.datetime(2016, 1, 1)
hoje = datetime.datetime.now()


end_date = hoje - datetime.timedelta(1)

In [6]:
#Consulta dados
all_data = get(tickers, start_date, end_date)

In [7]:
all_data.head()

High   Low  Open  Close      Volume  Adj Close
Ticker   Date                                                      
GGBR4.SA 2016-01-04  4.67  4.51  4.60   4.54   5058600.0   4.274928
         2016-01-05  4.61  4.33  4.55   4.33   6887400.0   4.077188
         2016-01-06  4.33  3.98  4.33   3.98  10245800.0   3.747624
         2016-01-07  3.90  3.70  3.90   3.80  12284300.0   3.578134
         2016-01-08  4.05  3.89  3.92   3.93  10692400.0   3.700544

In [8]:
all_data.tail()

High        Low  ...       Volume  Adj Close
Ticker   Date                              ...                        
BBDC4.SA 2020-11-09  23.260000  21.860001  ...  119615700.0  22.940001
         2020-11-10  24.620001  23.020000  ...  135808900.0  24.420000
         2020-11-11  24.590000  23.620001  ...   76246400.0  24.320000
         2020-11-12  24.320000  23.209999  ...   54954900.0  23.360001
         2020-11-13  24.100000  23.330000  ...   30707700.0  24.010000

[5 rows x 6 columns]

Analisar renatabilidade da carteia

Analise grafica das acoes

In [9]:
#Plotar dados como candlestic
trace = go.Candlestick(x=all_data.loc['BBDC4.SA'].index,
                       open=all_data.loc['BBDC4.SA'].Open,
                       high=all_data.loc['BBDC4.SA'].High,
                       low=all_data.loc['BBDC4.SA'].Low,
                       close=all_data.loc['BBDC4.SA'].Close)

data = [trace]

In [10]:
py.iplot(data)

In [11]:
data = all_data.reset_index()
data = data.set_index(['Date', 'Ticker']).sort_index()
close = data['Close']

#Gerar variavel com todos os dias da semana do dataset analisado
all_weekdays = pd.date_range(start=start_date, end=end_date, freq='B')

#Usar variavel gerada para reindexar os dados
#Preencher dias sem movimento com NaN (not a number - numpy)
close = close.reindex(pd.MultiIndex.from_product([all_weekdays, tickers],
                                                 names=['Date', 'Ticker']),
                      fill_value=np.NaN)

close.tail(12)

Date        Ticker  
2020-11-10  GGBR4.SA    20.830000
            CIEL3.SA     3.750000
            BBDC4.SA    24.420000
2020-11-11  GGBR4.SA    20.930000
            CIEL3.SA     3.800000
            BBDC4.SA    24.320000
2020-11-12  GGBR4.SA    20.680000
            CIEL3.SA     3.690000
            BBDC4.SA    23.360001
2020-11-13  GGBR4.SA    20.920000
            CIEL3.SA     3.730000
            BBDC4.SA    24.010000
Name: Close, dtype: float64

In [12]:
close = close.reset_index().pivot(index='Date',
                                  columns='Ticker',
                                  values='Close')

close.head()

Ticker,BBDC4.SA,CIEL3.SA,GGBR4.SA
Date,,,
2016-01-01,NaN,NaN,NaN
2016-01-04,11.8958,22.368099,4.54
2016-01-05,11.9271,23.437500,4.33
2016-01-06,11.7581,23.798599,3.98
2016-01-07,11.5202,23.888901,3.80


In [13]:
data = all_data.pct_change()
data.loc['BBDC4.SA'].Close.head()

Date
2016-01-04    2.189222
2016-01-05    0.002631
2016-01-06   -0.014169
2016-01-07   -0.020233
2016-01-08   -0.019566
Name: Close, dtype: float64

In [14]:
log_returns = np.log(close.dropna()).diff()
log_returns.head()

Ticker,BBDC4.SA,CIEL3.SA,GGBR4.SA
Date,,,
2016-01-04,NaN,NaN,NaN
2016-01-05,0.002628,0.046702,-0.047359
2016-01-06,-0.014271,0.015289,-0.084286
2016-01-07,-0.020440,0.003787,-0.046281
2016-01-08,-0.019760,0.017293,0.033638


In [15]:
layout = go.Layout(
    title='Result',
    yaxis=dict(
        title='Cumulative log returns',
        titlefont=dict(
            size=18,
            color='#7f7f7f'
        )
    )
)
axis = []

**Análise gráfica do retorno das ações analisadas**

In [16]:
for d in log_returns:
  axi = go.Scatter(x=log_returns.index,
                   y=log_returns[d].cumsum(),
                   name=d,
                   opacity=1)
  axis.append(axi)

fig = dict(data=axis, layout=layout)

py.iplot(fig)

In [17]:
layout2 = go.Layout(
    title='Result',
    yaxis=dict(
        title='Total relative returns (%)',
        titlefont=dict(
            size=18,
            color='#7f7f7f'
            )
        )
)
axis2 = []

In [18]:
for d in log_returns:
  axi2 = go.Scatter(
      x=log_returns.index,
      y=100 * (np.exp(log_returns[d].cumsum()) - 1),
      name=d,
      opacity=1
  )
  axis2.append(axi2)

fig2 = dict(data=axis2, layout=layout2)
py.iplot(fig2)

In [19]:
bbdc = all_data.loc['BBDC4.SA'].dropna()
bbdc = bbdc['2019-06-01':]
window = 21
MA = bbdc.Close.rolling(window=window).mean()

In [20]:
trace_avg = go.Scatter(x=MA.index,
                       y=MA,
                       name='BBDC4.SA MA(21)',
                       line=dict(color='#BEBECF'),
                       opacity=1)

trace_candles = go.Candlestick(x=bbdc.index,
                               open=bbdc.Open,
                               high=bbdc.High,
                               low=bbdc.Low,
                               close=bbdc.Close,
                               name='Price')

data = [trace_avg, trace_candles]
fig = dict(data=data)
py.iplot(fig)

**Cálculo e análise de média móvel dos preços das ações**
* Cálculos baseados no preço de fechamento das ações

In [21]:
window = 21
K = (2 / (window + 1))
bbdc = all_data.loc['BBDC4.SA'].dropna()
bbdc = bbdc['2019-06-01':]
MA = bbdc.Close.rolling(window=window).mean().dropna()

In [22]:
ema_data = pd.DataFrame(index=MA.index)
ema_data['Price'] = bbdc.Close
ema_data['MA'] = MA
ema_data['EMA'] = np.NaN

In [23]:
ema_data.head()

,Price,MA,EMA
Date,,,
2019-07-02,37.730000,37.104762,NaN
2019-07-03,38.400002,37.177143,NaN
2019-07-04,39.070000,37.273334,NaN
2019-07-05,39.410000,37.412857,NaN
2019-07-08,39.230000,37.505715,NaN


In [24]:
ema_data.EMA[0] = ema_data.MA[1]
ema_data.head()

,Price,MA,EMA
Date,,,
2019-07-02,37.730000,37.104762,37.177143
2019-07-03,38.400002,37.177143,NaN
2019-07-04,39.070000,37.273334,NaN
2019-07-05,39.410000,37.412857,NaN
2019-07-08,39.230000,37.505715,NaN


In [25]:
for i in range(1, len(ema_data)):
  ema_data.EMA[i] = (ema_data.Price[i] * K) + ((1 - K) * ema_data.EMA[i-1])

In [26]:
ema_data.head()

,Price,MA,EMA
Date,,,
2019-07-02,37.730000,37.104762,37.177143
2019-07-03,38.400002,37.177143,37.288312
2019-07-04,39.070000,37.273334,37.450284
2019-07-05,39.410000,37.412857,37.628440
2019-07-08,39.230000,37.505715,37.774036


In [27]:
trace_ma = go.Scatter(x=ema_data.index,
                      y=ema_data.MA,
                      name='BBDC4.SA MA(21)',
                      line=dict(color='#BEBECF'),
                      opacity=1)

trace_ema = go.Scatter(x=ema_data.index,
                       y=ema_data.EMA,
                       name='BBDC4.SA EMA(21)',
                       line=dict(color='#17BECF'),
                       opacity=1)

trace_candles = go.Candlestick(x=bbdc.index,
                               open=bbdc.Open,
                               high=bbdc.High,
                               low=bbdc.Low,
                               close=bbdc.Close,
                               name='Price')

data = [trace_ma, trace_ema, trace_candles]
fig = dict(data=data)
py.iplot(fig)

**Cruzamento de médias móveis**

No gráfico a seguir são comparadas as médias móveis de curto e longo prazo, como forma de subsidiar decisão de compra ou venda dos papéis.
* **Compra**: Média curta cruza a média longa para cima
* **Venda**: Média curta cruza a média longa para baixo

In [28]:
#Cruzamento de medias
bbdc = close.loc[:, 'BBDC4.SA'].dropna()
short_rolling_bbdc = bbdc.rolling(window=9).mean()
long_rolling_bbdc = bbdc.rolling(window=21).mean()

In [29]:
trace_short = go.Scatter(x=short_rolling_bbdc.index,
                         y=short_rolling_bbdc,
                         name='BBDC4.SA Short',
                         line=dict(color='#17BECF'),
                         opacity=1)

trace_long = go.Scatter(x=long_rolling_bbdc.index,
                         y=long_rolling_bbdc,
                         name='BBDC4.SA Long',
                         line=dict(color='#7F7F7F'),
                         opacity=1)

trace_price = go.Scatter(x=bbdc.index,
                         y=bbdc,
                         name='BBDC4.SA',
                         line=dict(color='#B22222'),
                         opacity=0.8)

data = [trace_short, trace_long, trace_price]

In [30]:
fig = dict(data=data)
py.iplot(fig)